# Network Query Agent - Simple Notebook Usage

Convert natural language questions to Cypher queries using Google Gemini AI.
Get reasoning and queries in clean JSON format.

In [ ]:
import json
from pathlib import Path

path = Path(r"C:\Matin\codes\AIOps\graph\snapshots\network_2026-01-04T12-35-25.341561.json")
data = json.loads(path.read_text())

# Build interface lookup: (hostname, interface_name) -> interface dict
iface_by_name = {}
for device in data.get("devices", []):
    hostname = device.get("hostname", "")
    for iface in device.get("interfaces", []):
        iface_by_name[(hostname, iface.get("interface", ""))] = iface

links = []
for device in data.get("devices", []):
    hostname = device.get("hostname", "")
    for cdp in device.get("cdp_neighbors", []):
        local_name = cdp.get("local_interface", "")
        neighbor_device = cdp.get("neighbor_device", "").split(".")[0]
        neighbor_name = cdp.get("neighbor_interface", "")
        if not local_name or not neighbor_device or not neighbor_name:
            continue

        local_iface = iface_by_name.get((hostname, local_name), {})
        remote_iface = iface_by_name.get((neighbor_device, neighbor_name), {})

        links.append({
            "local_device": hostname,
            "local_interface": local_name,
            "remote_device": neighbor_device,
            "remote_interface": neighbor_name,
            "neighbor_ip": cdp.get("neighbor_ip", ""),
            "local_ok": local_iface.get("ok", ""),
            "local_method": local_iface.get("method", ""),
            "local_status": local_iface.get("status", ""),
            "local_protocol": local_iface.get("protocol", ""),
            "remote_ok": remote_iface.get("ok", ""),
            "remote_method": remote_iface.get("method", ""),
            "remote_status": remote_iface.get("status", ""),
            "remote_protocol": remote_iface.get("protocol", ""),
        })

links[:]


[{'local_device': 'EDGE-R1',
  'local_interface': 'GigabitEthernet0/0',
  'remote_device': 'MANAGEMENT',
  'remote_interface': 'GigabitEthernet0/1',
  'neighbor_ip': '10.10.10.10',
  'local_ok': 'YES',
  'local_method': 'NVRAM',
  'local_status': 'up',
  'local_protocol': 'up',
  'remote_ok': 'YES',
  'remote_method': 'unset',
  'remote_status': 'up',
  'remote_protocol': 'up'},
 {'local_device': 'MANAGEMENT',
  'local_interface': 'GigabitEthernet0/3',
  'remote_device': 'CORE-SW2',
  'remote_interface': 'GigabitEthernet0/1',
  'neighbor_ip': '10.10.10.3',
  'local_ok': 'YES',
  'local_method': 'unset',
  'local_status': 'up',
  'local_protocol': 'up',
  'remote_ok': 'YES',
  'remote_method': 'unset',
  'remote_status': 'up',
  'remote_protocol': 'up'},
 {'local_device': 'MANAGEMENT',
  'local_interface': 'GigabitEthernet0/2',
  'remote_device': 'CORE-SW1',
  'remote_interface': 'GigabitEthernet0/1',
  'neighbor_ip': '10.10.10.2',
  'local_ok': 'YES',
  'local_method': 'unset',
  'loca

In [ ]:
# Network Query Agent - Interactive Demo
# Run this cell to start asking questions!

import sys
from pathlib import Path
sys.path.append(str(Path().absolute()))

from agents.query_agent import NetworkQueryAgent

# Initialize agent
agent = NetworkQueryAgent()

# Ask a question
question = "Show me all devices"
result = agent.ask(question)

# Display results
if 'error' not in result:
    print(f"\n✅ Found {result['count']} results:\n")
    for i, record in enumerate(result['results'], 1):
        print(f"{i}. {record}")
else:
    print(f"❌ Error: {result['error']}")

# Don't forget to close when done
# agent.close()


c:\Matin\codes\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



NETWORK QUERY AGENT

[QUESTION] Show me all devices

[REASONING]
1. Understanding: User wants to see all network devices
2. Node Types: Device nodes only
3. Relationships: None needed
4. Properties: hostname, type, ip_address (from Device schema)
5. Query Plan: Simple MATCH on Device nodes, return key properties

[CYPHER] MATCH (d:Device) RETURN d.hostname, d.type, d.ip_address LIMIT 100

[EXECUTING]...
[OK] Found 6 results

✅ Found 6 results:

1. {'d.hostname': 'EDGE-R1', 'd.type': 'router', 'd.ip_address': '10.10.10.1'}
2. {'d.hostname': 'MANAGEMENT', 'd.type': 'switch', 'd.ip_address': '10.10.10.10'}
3. {'d.hostname': 'CORE-SW1', 'd.type': 'switch', 'd.ip_address': '10.10.10.2'}
4. {'d.hostname': 'CORE-SW2', 'd.type': 'switch', 'd.ip_address': '10.10.10.3'}
5. {'d.hostname': 'ACC-SW1', 'd.type': 'switch', 'd.ip_address': '10.10.10.4'}
6. {'d.hostname': 'ACC-SW2', 'd.type': 'switch', 'd.ip_address': '10.10.10.5'}


In [6]:
# Network Query Agent - Display Results Properly
import sys
from pathlib import Path
sys.path.append(str(Path().absolute()))

from agents import NetworkQueryAgent
import json

# Initialize agent
agent = NetworkQueryAgent()

# Test different queries
result = agent.ask("show up interfaces of CORE-SW2", deduplicate=True)

# Display header
print(f"Template: {result['template']}")
print(f"Cypher: {result['cypher']}")
print(f"\nFound {result['count']} results:\n")

# Display results properly
for i, record in enumerate(result['results'], 1):
    print(f"{i}. {record}")

# Close
agent.close()


Template: show_up_interfaces_device
Cypher: MATCH (d:Device {hostname: "CORE-SW2"})-[:HAS_INTERFACE]->(i:Interface)
WHERE i.status = "up" AND i.protocol = "up"
RETURN i.name AS iface, i.ip_address AS ip
ORDER BY iface

Found 7 results:

1. {'iface': 'GigabitEthernet0/1', 'ip': 'unassigned'}
2. {'iface': 'GigabitEthernet0/2', 'ip': 'unassigned'}
3. {'iface': 'GigabitEthernet0/3', 'ip': 'unassigned'}
4. {'iface': 'GigabitEthernet1/0', 'ip': 'unassigned'}
5. {'iface': 'GigabitEthernet1/1', 'ip': 'unassigned'}
6. {'iface': 'Port-channel1', 'ip': 'unassigned'}
7. {'iface': 'Vlan10', 'ip': '10.10.10.3'}


In [4]:
# Display results properly (hostname + ip_address for paths)
for i, record in enumerate(result['results'], 1):
    path = record.get('p')  # "show_all_paths" returns p
    if isinstance(path, dict) and 'nodes' in path:
        nodes = []
        for node in path['nodes']:
            props = node.get('properties', {})
            hostname = props.get('hostname', 'unknown')
            ip = props.get('ip_address', '')
            nodes.append(f"{hostname} ({ip})" if ip else hostname)
        print(f"{i}. " + " -> ".join(nodes))
    else:
        print(f"{i}. {record}")


1. EDGE-R1 (10.10.10.1) -> unknown (unassigned) -> unknown (unassigned) -> MANAGEMENT (10.10.10.10) -> unknown (unassigned) -> unknown (unassigned) -> CORE-SW1 (10.10.10.2) -> unknown (unassigned) -> unknown (unassigned) -> ACC-SW1 (10.10.10.4)


In [2]:
from agents import SnapshotManager, NetworkQueryAgent

# 1. List snapshots
snapshot_mgr = SnapshotManager()
snapshots = snapshot_mgr.display_snapshots()


c:\Matin\codes\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Matin\codes\myenv\Lib\site-packages\langchain_core\_api\deprecation.py:26: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1



AVAILABLE NETWORK SNAPSHOTS
#    Timestamp            Devices    Size       Filename                      
--------------------------------------------------------------------------------
1    2026-01-04 14:48:23  4          24.7 KB    network_2026-01-04T14-48-23.641220.json
2    2026-01-04 14:41:36  6          43.5 KB    network_2026-01-04T14-41-36.944110.json
3    2026-01-04 14:15:30  6          44.1 KB    network_2026-01-04T14-15-30.785182.json


In [3]:

# Output:
# ================================================================================
# AVAILABLE NETWORK SNAPSHOTS
# ================================================================================
# #    Timestamp            Devices    Size       Filename
# --------------------------------------------------------------------------------
# 1    2026-01-04 14:48:23  6          12.3 KB    network_2026-01-04T14-48-23.641220.json
# 2    2026-01-04 14:41:36  6          12.1 KB    network_2026-01-04T14-41-36.944110.json
# 3    2026-01-04 14:15:30  6          12.0 KB    network_2026-01-04T14-15-30.785182.json
# ================================================================================

# 2. Load a snapshot
snapshot_mgr.clear_neo4j()
result = snapshot_mgr.load_snapshot(snapshot_index=1)

# 3. Query the network
agent = NetworkQueryAgent()
result = agent.ask("Show me all devices")


# 4. Load a different snapshot
snapshot_mgr.clear_neo4j()
snapshot_mgr.load_snapshot(snapshot_index=2)
# Now ask questions about the new snapshot!

# 5. Cleanup
agent.close()
snapshot_mgr.close()



[WARNING] Clearing all data from Neo4j...
[OK] Neo4j database cleared

[LOADING] network_2026-01-04T14-48-23.641220.json

FEEDING SNAPSHOT TO NEO4J
Snapshot ID: 2026-01-04T14:48:23.641220
Devices: 4

[PHASE 0] Creating Snapshot Node
[OK] Snapshot node created

[PHASE 1] Creating Device Nodes
[OK] Created 4 devices

[PHASE 2] Creating Interfaces
[OK] Created 57 interfaces

[PHASE 3] Storing Switch Extra Data
[OK] Extra data stored

[PHASE 4] Building Lookups
[OK] Indexed 57 interfaces, 4 devices

[PHASE 5] Creating CDP Connections
[OK] Created 1 CDP connections

[PHASE 6] Creating OSPF Connections
[OK] Created 0 OSPF connections

[SUCCESS] SNAPSHOT LOADED INTO NEO4J
  - Devices: 4
  - Interfaces: 57
  - CDP connections: 1
  - OSPF connections: 0

[WARNING] Clearing all data from Neo4j...
[OK] Neo4j database cleared

[LOADING] network_2026-01-04T14-41-36.944110.json

FEEDING SNAPSHOT TO NEO4J
Snapshot ID: 2026-01-04T14:41:36.944110
Devices: 6

[PHASE 0] Creating Snapshot Node
[OK] Snaps

In [ ]:
from agents.cli_agent import NetworkCLIAgent

agent = NetworkCLIAgent()

# Example request
result = agent.ask("Show VLANs on CORE-SW1 port 5012")

print(result)
